In [1]:
!pip install polars --no-index --find-links=file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg
!pip install duckdb --no-index --find-links=file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg

Looking in links: file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg
Processing /kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg/polars-0.20.16-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Looking in links: file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg
Processing /kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg/duckdb-0.8.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


In [2]:
import polars as pl
import duckdb as dd
import os
from datetime import datetime
from joblib import Parallel, delayed
from tqdm import tqdm
from pathlib import Path

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

import tensorflow as tf
# import tensorflow_io as tfio
from tensorflow import keras

E0000 00:00:1730470801.314294      13 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:479
D1101 14:20:01.322762174      13 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D1101 14:20:01.322775799      13 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D1101 14:20:01.322779064      13 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D1101 14:20:01.322781443      13 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D1101 14:20:01.322783857      13 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defa

In [3]:
# detect TPUs
tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='local')
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

print("Number of accelerators: ", tpu_strategy.num_replicas_in_sync)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1730470816.079345      13 service.cc:145] XLA service 0x596116d934e0 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730470816.079399      13 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1730470816.079404      13 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1730470816.079407      13 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1730470816.079410      13 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1730470816.079412      13 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1730470816.079415      13 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1730470816.079418      13 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1730470816.079420      13 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)
I

In [4]:
from multiprocessing import cpu_count
n_cores = cpu_count()
print(f'Number of Logical CPU cores: {n_cores}')

Number of Logical CPU cores: 96


In [5]:
Test = True

if Test:
    num_jobs = 92
else:
    num_jobs = 4

config_dict = {'train_csv_path': '/kaggle/input/child-mind-institute-problematic-internet-use/train.csv'}
config_dict['test_csv_path'] = '/kaggle/input/child-mind-institute-problematic-internet-use/test.csv'
config_dict['train_parquet_root_path'] = '/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/'
config_dict['test_parquet_root_path'] = '/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet/'

In [6]:
train_data = pl.read_csv(config_dict['train_csv_path'], low_memory=True)
# train_data.describe()
train_data_id_list = pl.Series(train_data.select(pl.col('id'))).to_list()
# len(train_data_id_list)

# actigraphy_file_list = []
actigraphy_id_list = []

for dirname, _, filenames in os.walk(config_dict['train_parquet_root_path']):
    for filename in filenames:
        full_parquet_file_path = os.path.join(dirname, filename)
        # actigraphy_file_list.append(full_parquet_file_path)
        actigraphy_id_list.append(full_parquet_file_path.split('/')[-2].replace('id=',''))
        
final_id_list = list(set(actigraphy_id_list).intersection(train_data_id_list))

print(len(final_id_list))
print(len(train_data_id_list))

996
3960


In [7]:
categorical_colms = ['Basic_Demos-Enroll_Season','CGAS-Season','Physical-Season','Fitness_Endurance-Season','FGC-Season'\
                    ,'BIA-Season','PCIAT-Season','PreInt_EduHx-Season','PAQ_A-Season','PAQ_C-Season','SDS-Season']

for cat_col in categorical_colms:
    cat_vals_list = pl.Series(train_data.select(pl.col(cat_col))).to_list()
    encoded_vals_list = le.fit_transform(cat_vals_list)
    encoded_col = "encoded_{0}".format(cat_col)
    train_data = train_data.with_columns(pl.Series(name=encoded_col, values=encoded_vals_list))
    train_data = train_data.drop([cat_col])

In [8]:
# Calculate the approximate size of each smaller list
chunk_size = len(train_data_id_list) // num_jobs
remainder = len(train_data_id_list) % num_jobs

# Create the num_jobs lists
result = []
start = 0

for i in range(num_jobs):
    # Add an extra element to the first few lists to account for remainder
    end = start + chunk_size + (1 if i < remainder else 0)
    result.append(train_data_id_list[start:end])
    start = end

# Check the result
"""for i, sublist in enumerate(result):
    print(f"List {i + 1} (length {len(sublist)}): {sublist}")"""

'for i, sublist in enumerate(result):\n    print(f"List {i + 1} (length {len(sublist)}): {sublist}")'

In [9]:
def get_hours(time_of_day):
    ts = int(time_of_day)
    ts /= 1000
    date_var = datetime.utcfromtimestamp(ts).strftime('%H:%M:%S')
    return int(date_var.split(':')[0])

def get_mins(time_of_day):
    ts = int(time_of_day)
    ts /= 1000
    date_var = datetime.utcfromtimestamp(ts).strftime('%H:%M:%S')
    return int(date_var.split(':')[1])
    
def get_secs(time_of_day):
    ts = int(time_of_day)
    ts /= 1000
    date_var = datetime.utcfromtimestamp(ts).strftime('%H:%M:%S')
    return int(date_var.split(':')[2])

In [10]:
def transform_parquet_file(parquet_file_path, file_id):

    directory = Path(parquet_file_path)

    # Check if any Parquet file exists in the directory
    parquet_files = list(directory.glob("*.parquet"))

    if parquet_files:
        parquet_df = pl.read_parquet(parquet_file_path+"*.parquet")
        parquet_df = parquet_df.drop(['id'])
        parquet_df = parquet_df.with_columns([
            pl.lit(file_id).alias("id"),
            pl.col("time_of_day").map_elements(get_hours, return_dtype=pl.Int32).alias("hour_of_day")
        ])

        parquet_df = parquet_df.drop(['time_of_day'])
        parquet_df = parquet_df.group_by(["id","weekday","hour_of_day","relative_date_PCIAT"])\
        .agg([pl.mean("enmo").name.suffix("_avg"), pl.mean("anglez").name.suffix("_avg"), pl.mean("light").name.suffix("_avg")\
              , pl.mean("battery_voltage").name.suffix("_avg")])
    else:
        columns = [("id", pl.String), ("weekday", pl.Int8), ("hour_of_day", pl.Int32), ("relative_date_PCIAT", pl.Float32)\
                  , ("enmo_avg", pl.Float32), ("anglez_avg", pl.Float32), ("light_avg", pl.Float32)\
                   , ("battery_voltage_avg", pl.Float32)]

        parquet_df = pl.DataFrame(schema=columns)
    
    return parquet_df

In [11]:
def unify_files(list_of_ids):
    df_initializing_id = list_of_ids.pop()
    parquet_file_name = "id={0}/".format(df_initializing_id)
    first_parquet_file = config_dict['train_parquet_root_path']+parquet_file_name
    first_parquet_df = transform_parquet_file(first_parquet_file, df_initializing_id)
    
    train_data_of_id = train_data.filter(pl.col('id')==df_initializing_id)
    first_joined_features_df = train_data_of_id.join(first_parquet_df, on="id", how="left")
    
    for id in tqdm(list_of_ids):
        parquet_file_name = "id={0}/".format(id)
        parquet_file_path = config_dict['train_parquet_root_path']+parquet_file_name
        parquet_df = transform_parquet_file(parquet_file_path, id)
        
        train_data_of_id = train_data.filter(pl.col('id')==id)
        joined_features = train_data_of_id.join(parquet_df, on="id", how="left")
        
        try:
            first_joined_features_df = pl.concat([first_joined_features_df, joined_features])
        except:
            print(id)
    return first_joined_features_df

In [12]:
parts_of_parquet = (Parallel(n_jobs=num_jobs)(delayed(unify_files)(id_list) for id_list in result))

100%|██████████| 42/42 [00:59<00:00,  1.41s/it]


In [13]:
len(parts_of_parquet)

92

In [14]:
for i in range(len(parts_of_parquet)):
    print(parts_of_parquet[i].shape)

(3204, 89)
(7472, 89)
(7629, 89)
(7901, 89)
(6563, 89)
(9645, 89)
(7501, 89)
(4982, 89)
(7391, 89)
(3209, 89)
(4777, 89)
(5766, 89)
(8381, 89)
(7686, 89)
(7451, 89)
(6142, 89)
(8901, 89)
(7535, 89)
(5613, 89)
(8417, 89)
(6893, 89)
(6530, 89)
(7654, 89)
(8267, 89)
(6475, 89)
(9323, 89)
(3563, 89)
(8536, 89)
(8287, 89)
(6617, 89)
(6495, 89)
(4962, 89)
(7865, 89)
(2749, 89)
(5208, 89)
(7309, 89)
(7299, 89)
(4920, 89)
(5554, 89)
(6693, 89)
(7078, 89)
(6957, 89)
(3612, 89)
(5554, 89)
(3134, 89)
(4511, 89)
(6146, 89)
(5713, 89)
(6098, 89)
(7340, 89)
(7215, 89)
(5289, 89)
(7420, 89)
(4643, 89)
(9225, 89)
(3226, 89)
(5378, 89)
(9822, 89)
(7641, 89)
(8201, 89)
(8717, 89)
(7198, 89)
(6719, 89)
(7961, 89)
(8219, 89)
(4340, 89)
(4358, 89)
(4449, 89)
(6806, 89)
(5457, 89)
(7838, 89)
(6925, 89)
(8697, 89)
(7163, 89)
(7550, 89)
(2874, 89)
(4497, 89)
(5619, 89)
(6318, 89)
(3034, 89)
(6671, 89)
(6251, 89)
(8525, 89)
(4927, 89)
(6817, 89)
(5993, 89)
(6336, 89)
(6927, 89)
(9548, 89)
(7720, 89)
(7501, 89)

In [15]:
test_df = parts_of_parquet[0]
#dd.sql("select * from test_df limit 100").pl()

In [30]:
from sklearn.preprocessing import StandardScaler

test_df_pandas = test_df.to_pandas()
complete_column = "id"

missing_mask = test_df_pandas.drop(columns=[complete_column]).isna()

for col in missing_mask.columns:
    test_df_pandas[col] = test_df_pandas[col].fillna(test_df_pandas[col].mean())

test_df_pandas.head(100)

,id,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,...,encoded_PAQ_A-Season,encoded_PAQ_C-Season,encoded_SDS-Season,weekday,hour_of_day,relative_date_PCIAT,enmo_avg,anglez_avg,light_avg,battery_voltage_avg
0,023eba7b,16,0,61.940529,21.937701,66.5,138.0,31.000000,70.000000,99.000000,...,3,4,4,3.980101,11.499368,56.162666,0.026015,-8.368336,28.082163,3868.136475
1,00008ff9,5,0,51.000000,16.877316,46.0,50.8,26.846154,63.538413,82.602194,...,4,4,4,3.980101,11.499368,56.162666,0.026015,-8.368336,28.082163,3868.136475
2,000fd460,9,0,61.940529,14.035590,48.0,46.0,22.000000,75.000000,70.000000,...,4,0,0,3.980101,11.499368,56.162666,0.026015,-8.368336,28.082163,3868.136475
3,00105258,10,1,71.000000,16.648696,56.5,75.6,26.846154,65.000000,94.000000,...,4,2,0,3.980101,11.499368,56.162666,0.026015,-8.368336,28.082163,3868.136475
4,00115b9f,9,0,71.000000,18.292347,56.0,81.6,26.846154,60.000000,97.000000,...,4,3,2,6.000000,0.000000,57.000000,0.000129,-51.537354,7.008241,3994.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,00115b9f,9,0,71.000000,18.292347,56.0,81.6,26.846154,60.000000,97.000000,...,4,3,2,3.000000,11.000000,68.000000,0.035037,5.601849,3.836386,3935.000000
96,00115b9f,9,0,71.000000,18.292347,56.0,81.6,26.846154,60.000000,97.000000,...,4,3,2,6.000000,11.000000,78.000000,0.022498,63.321850,2.678532,3859.000000
97,00115b9f,9,0,71.000000,18.292347,56.0,81.6,26.846154,60.000000,97.000000,...,4,3,2,3.000000,14.000000,82.000000,0.029208,-38.547047,6.117274,3839.068848
98,00115b9f,9,0,71.000000,18.292347,56.0,81.6,26.846154,60.000000,97.000000,...,4,3,2,4.000000,12.000000,83.000000,0.024809,31.689722,9.646648,3835.088135


In [35]:
import pandas as pd
scaler = StandardScaler()

df_scaled_values = scaler.fit_transform(test_df_pandas.drop(columns=[complete_column]))
df_scaled = pd.DataFrame(df_scaled_values, columns=test_df_pandas.drop(columns=[complete_column]).columns)

# Add the complete column back to the scaled DataFrame
df_scaled[complete_column] = test_df_pandas[complete_column].values
df_scaled = df_scaled[test_df_pandas.columns]  # Ensure original column order

df_scaled.head(100)

,id,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,...,encoded_PAQ_A-Season,encoded_PAQ_C-Season,encoded_SDS-Season,weekday,hour_of_day,relative_date_PCIAT,enmo_avg,anglez_avg,light_avg,battery_voltage_avg
0,023eba7b,1.884152,-0.473112,9.546355e-16,-0.033265,1.724340,0.748998,1.909039e+01,1.268161,1.684483e+00,...,-10.303424,1.268610,1.769989,0.000000,0.000000,1.937035e-08,3.903569e-08,-2.367866e-08,-1.390235e-09,8.134315e-08
1,00008ff9,-2.028241,-0.473112,-1.469893e+00,-0.893555,-3.604611,-1.904877,-1.632769e-14,0.000000,1.459826e-15,...,0.038735,1.268610,1.769989,0.000000,0.000000,1.937035e-08,3.903569e-08,-2.367866e-08,-1.390235e-09,8.134315e-08
2,000fd460,-0.605553,-0.473112,9.546355e-16,-1.376662,-3.084713,-2.050962,-2.227213e+01,2.249470,-1.294575e+00,...,0.038735,-1.610530,-1.460455,0.000000,0.000000,1.937035e-08,3.903569e-08,-2.367866e-08,-1.390235e-09,8.134315e-08
3,00105258,-0.249881,2.113664,1.217167e+00,-0.932422,-0.875148,-1.150105,-1.632769e-14,0.286853,1.170852e+00,...,0.038735,-0.170960,-1.460455,0.000000,0.000000,1.937035e-08,3.903569e-08,-2.367866e-08,-1.390235e-09,8.134315e-08
4,00115b9f,-0.605553,-0.473112,1.217167e+00,-0.652993,-1.005123,-0.967499,-1.632769e-14,-0.694455,1.479031e+00,...,0.038735,0.548825,0.154767,1.032355,-1.675712,3.306523e-02,-1.017258e+00,-9.796965e-01,-3.649979e-01,6.855156e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,00115b9f,-0.605553,-0.473112,1.217167e+00,-0.652993,-1.005123,-0.967499,-1.632769e-14,-0.694455,1.479031e+00,...,0.038735,0.548825,0.154767,-0.500922,-0.072769,4.674409e-01,3.545620e-01,3.170454e-01,-4.199341e-01,3.641722e-01
96,00115b9f,-0.605553,-0.473112,1.217167e+00,-0.652993,-1.005123,-0.967499,-1.632769e-14,-0.694455,1.479031e+00,...,0.038735,0.548825,0.154767,1.032355,-0.072769,8.623278e-01,-1.381820e-01,1.626968e+00,-4.399880e-01,-4.976171e-02
97,00115b9f,-0.605553,-0.473112,1.217167e+00,-0.652993,-1.005123,-0.967499,-1.632769e-14,-0.694455,1.479031e+00,...,0.038735,0.548825,0.154767,-0.500922,0.364397,1.020283e+00,1.254723e-01,-6.848888e-01,-3.804293e-01,-1.583167e-01
98,00115b9f,-0.605553,-0.473112,1.217167e+00,-0.652993,-1.005123,-0.967499,-1.632769e-14,-0.694455,1.479031e+00,...,0.038735,0.548825,0.154767,0.010170,0.072953,1.059771e+00,-4.739877e-02,9.090950e-01,-3.193010e-01,-1.799976e-01


In [36]:
# Parameters for the MICE-like imputation
iterations = 17  # Number of iterations for imputation
epochs = 100  # Training epochs for each imputer model
batch_size = 16 * tpu_strategy.num_replicas_in_sync  # Batch size for training

# Function to create a simple neural network model for imputation
def create_imputer_model(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(100, activation="relu", input_shape=(input_dim,)),
        tf.keras.layers.Dense(85, activation="relu"),
        tf.keras.layers.Dense(65, activation="relu"),
        tf.keras.layers.Dense(35, activation="relu"),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer="adamax", loss="mean_squared_error")
    return model

In [50]:
import numpy as np

with tpu_strategy.scope():
    for iteration in range(iterations):
        print(f"\nIteration {iteration + 1}/{iterations}")

        # Iterate over each column to impute, excluding the complete column
        for col in df_scaled.columns:
            print("Starting with column --> {0}".format(col))

            if col == complete_column:
                print("complete column; skipping over")
                continue  # Skip the complete column

            # Use the original missing mask to find rows with missing values
            missing_rows = missing_mask[col]

            # If no missing values, skip this column
            if not missing_rows.any():
                print("no missing data; skipping over")
                continue

            print("training for column --> {0}".format(col))
            # Define training data using columns other than the target column
            X_train = df_scaled.loc[~missing_rows].drop(columns=[col, complete_column]).values
            y_train = df_scaled.loc[~missing_rows, col].values

            # Define data to be predicted (rows where target column has missing values)
            X_pred = df_scaled.loc[missing_rows].drop(columns=[col, complete_column]).values

            X_train = np.array(X_train, dtype=np.float32)
            y_train = np.array(y_train, dtype=np.float32)
            X_pred = np.array(X_pred, dtype=np.float32)

            # Create and train the imputer model
            imputer_model = create_imputer_model(input_dim=X_train.shape[1])
            imputer_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

            # Predict missing values and update the DataFrame
            predicted_values = imputer_model.predict(X_pred).flatten()
            df_scaled.loc[missing_rows, col] = predicted_values


Iteration 1/17
Starting with column --> id
complete column; skipping over
Starting with column --> Basic_Demos-Age
no missing data; skipping over
Starting with column --> Basic_Demos-Sex
no missing data; skipping over
Starting with column --> CGAS-CGAS_Score
training for column --> CGAS-CGAS_Score


/usr/local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-11-01 15:04:19.711129: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node StatefulPartitionedCall.
I0000 00:00:1730473459.956182     859 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(d24346abc4149311:0:0), session_name()
I0000 00:00:1730473460.364629     859 tpu_compile_op_common.cc:245] Compilation of d24346abc4149311:0:0 with session name  took 408.406527ms and succeeded
I0000 00:00:1730473460.367795     859 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(d24346abc4149311:0:0), s

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 856ms/step


I0000 00:00:1730473515.666396     908 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(85804c023a0c4a7b:0:0), session_name()
I0000 00:00:1730473515.756172     908 tpu_compile_op_common.cc:245] Compilation of 85804c023a0c4a7b:0:0 with session name  took 89.731831ms and succeeded
I0000 00:00:1730473515.757146     908 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(85804c023a0c4a7b:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_data_distributed_17185690794720253171", property.function_library_fingerprint = 3249260976836082871, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "4,87,;", property.guaranteed_constants_size = 0, embedding_

Starting with column --> Physical-BMI
training for column --> Physical-BMI


I0000 00:00:1730473519.257417     854 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(a06a2b99333b8f68:0:0), session_name()
I0000 00:00:1730473519.646939     854 tpu_compile_op_common.cc:245] Compilation of a06a2b99333b8f68:0:0 with session name  took 389.482975ms and succeeded
I0000 00:00:1730473519.650210     854 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(a06a2b99333b8f68:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_iterator_7158594234667614164", property.function_library_fingerprint = 14287038548144303800, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "63,87,;63,;", property.guaranteed_constants_size = 0, embedding_pa

 3/17 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step  

2024-11-01 15:06:13.841244: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node sequential_6_1/dense_30_1/Add/ReadVariableOp.
I0000 00:00:1730473573.870072     872 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(dc71e40bb76de3e8:0:0), session_name()
I0000 00:00:1730473573.934670     872 tpu_compile_op_common.cc:245] Compilation of dc71e40bb76de3e8:0:0 with session name  took 64.561319ms and succeeded
I0000 00:00:1730473573.935205     872 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(dc71e40bb76de3e8:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_data_distributed_1475694961096069229", property.function_library_fingerprint = 6330706768104190395, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2,

15/17 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step

TypeError: '>' not supported between instances of 'int' and 'NoneType'

In [49]:
df_scaled.loc[missing_rows, col]

746     0.0
1325    0.0
1326    0.0
1327    0.0
1328    0.0
       ... 
3191    0.0
3193    0.0
3196    0.0
3201    0.0
3203    0.0
Name: Physical-BMI, Length: 541, dtype: float64